In [1]:
import pandas as pd
import numpy as np
import os

# Change this to your own DAVID API token
token = 'jiangjl@caltech.edu'

data_folder = '../data/thomsonlab_signaling/'
file_name = 'onmf_gene_list_20'

onmf_csv_path = data_folder + file_name + '.csv'
if not os.path.exists(data_folder + file_name + '_david'):
    os.mkdir(data_folder + file_name + '_david')

In [4]:
import logging
import traceback as tb
import suds.metrics as metrics
from tests import *
from suds import *
from suds.client import Client
from datetime import datetime

save_folder = data_folder + file_name + '_david/'
all_onmf_df = pd.read_csv(onmf_csv_path)
num_list = all_onmf_df.shape[1]
convert_table = pd.read_csv('convert_gene_id.txt', sep='\t')
gene_id_map = {entry['From']: entry['To'] for _, entry in convert_table.iterrows()}

In [5]:
def map_gene_id(gene_name):
    try:
        return gene_id_map[gene_name]
    except:
        return gene_name

overlap=3
initialSeed = 3
finalSeed = 3
linkage = 0.5
kappa = 50


errors = 0
setup_logging()
logging.getLogger('suds.client').setLevel(logging.DEBUG)
url = 'https://david.ncifcrf.gov/webservice/services/DAVIDWebService?wsdl'
print('url=%s' % url)

client = Client(url)
client.wsdl.services[0].setlocation('https://david.ncifcrf.gov/webservice/services/DAVIDWebService.DAVIDWebServiceHttpSoap11Endpoint/')
client.service.authenticate(token);


def parse_output(termClusteringReport, save_path):
    totalRows = len(termClusteringReport)
    print('Total clusters:',totalRows)
    resF = save_path
    with open(resF, 'w') as fOut:
        i = 0
        for simpleTermClusterRecord in termClusteringReport:
            i = i+1
            EnrichmentScore = simpleTermClusterRecord.score
            fOut.write('Annotation Cluster '+str(i) + '\tEnrichmentScore:'+str(EnrichmentScore)+'\n')
            fOut.write('Category\tTerm\tCount\t%\tPvalue\tGenes\tList Total\tPop Hits\tPop Total\tFold Enrichment\tBonferroni\tBenjamini\tFDR\n')
            for simpleChartRecord in  simpleTermClusterRecord.simpleChartRecords:
                    categoryName = simpleChartRecord.categoryName
                    termName = simpleChartRecord.termName
                    listHits = simpleChartRecord.listHits
                    percent = simpleChartRecord.percent
                    ease = simpleChartRecord.ease
                    Genes = simpleChartRecord.geneIds
                    listTotals = simpleChartRecord.listTotals
                    popHits = simpleChartRecord.popHits
                    popTotals = simpleChartRecord.popTotals
                    foldEnrichment = simpleChartRecord.foldEnrichment
                    bonferroni = simpleChartRecord.bonferroni
                    benjamini = simpleChartRecord.benjamini
                    FDR = simpleChartRecord.afdr
                    rowList = [categoryName,termName,str(listHits),str(percent),str(ease),Genes,str(listTotals),str(popHits),str(popTotals),str(foldEnrichment),str(bonferroni),str(benjamini),str(FDR)]
                    fOut.write('\t'.join(rowList)+'\n')


for ii in range(num_list):
    cur_gene_list = all_onmf_df[str(ii)]
    cur_gene_list = cur_gene_list[~cur_gene_list.isna()]
    cur_gene_list = cur_gene_list.apply(map_gene_id)
    save_path = save_folder + 'list_%d.csv' % ii

    inputIds = (',').join(cur_gene_list)
    idType = 'ENSEMBL_GENE_ID'
    listName = 'List_%d' % ii
    listType = 0

    client.service.addList(inputIds, idType, listName, listType);
    termClusteringReport = client.service.getTermClusterReport(overlap, initialSeed, finalSeed, linkage, kappa);
    parse_output(termClusteringReport, save_path);



url=https://david.ncifcrf.gov/webservice/services/DAVIDWebService?wsdl


2023-05-04 20:43:19,245 [DEBUG] send() @client.py:639
sending to (https://david.ncifcrf.gov/webservice/services/DAVIDWebService.DAVIDWebServiceHttpSoap11Endpoint/)
message:
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ns0="http://service.session.sample" xmlns:ns1="http://schemas.xmlsoap.org/soap/envelope/">
   <SOAP-ENV:Header/>
   <ns1:Body>
      <ns0:authenticate>
         <ns0:args0>jiangjl@caltech.edu</ns0:args0>
      </ns0:authenticate>
   </ns1:Body>
</SOAP-ENV:Envelope>

2023-05-04 20:43:19,246 [DEBUG] headers() @client.py:679
headers = {'Content-Type': 'text/xml; charset=utf-8', 'SOAPAction': '"urn:authenticate"'}

2023-05-04 20:43:19,608 [DEBUG] succeeded() @client.py:693
http succeeded:
b'<?xml version=\'1.0\' encoding=\'utf-8\'?><soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"><soapenv:Body><ns:authenticateResponse x

Total clusters: 26


2023-05-04 20:43:21,392 [DEBUG] succeeded() @client.py:693
http succeeded:
b'<?xml version=\'1.0\' encoding=\'utf-8\'?><soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"><soapenv:Body><ns:addListResponse xmlns:ns="http://service.session.sample"><ns:return>0.9342105263157895</ns:return></ns:addListResponse></soapenv:Body></soapenv:Envelope>'

2023-05-04 20:43:21,396 [DEBUG] send() @client.py:639
sending to (https://david.ncifcrf.gov/webservice/services/DAVIDWebService.DAVIDWebServiceHttpSoap11Endpoint/)
message:
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ns0="http://service.session.sample" xmlns:ns1="http://schemas.xmlsoap.org/soap/envelope/">
   <SOAP-ENV:Header/>
   <ns1:Body>
      <ns0:getTermClusterReport>
         <ns0:args0>3</ns0:args0>
         <ns0:args1>3</ns0:args1>
         <ns0:args2>3</ns0:args2>
         <ns0:args3

Total clusters: 15


...

Total clusters: 8


2023-05-04 20:43:47,151 [DEBUG] succeeded() @client.py:693
http succeeded:
b'<?xml version=\'1.0\' encoding=\'utf-8\'?><soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"><soapenv:Body><ns:addListResponse xmlns:ns="http://service.session.sample"><ns:return>0.9387755102040817</ns:return></ns:addListResponse></soapenv:Body></soapenv:Envelope>'

2023-05-04 20:43:47,155 [DEBUG] send() @client.py:639
sending to (https://david.ncifcrf.gov/webservice/services/DAVIDWebService.DAVIDWebServiceHttpSoap11Endpoint/)
message:
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ns0="http://service.session.sample" xmlns:ns1="http://schemas.xmlsoap.org/soap/envelope/">
   <SOAP-ENV:Header/>
   <ns1:Body>
      <ns0:getTermClusterReport>
         <ns0:args0>3</ns0:args0>
         <ns0:args1>3</ns0:args1>
         <ns0:args2>3</ns0:args2>
         <ns0:args3

Total clusters: 13


2023-05-04 20:43:48,443 [DEBUG] succeeded() @client.py:693
http succeeded:
b'<?xml version=\'1.0\' encoding=\'utf-8\'?><soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"><soapenv:Body><ns:addListResponse xmlns:ns="http://service.session.sample"><ns:return>1.0</ns:return></ns:addListResponse></soapenv:Body></soapenv:Envelope>'

2023-05-04 20:43:48,446 [DEBUG] send() @client.py:639
sending to (https://david.ncifcrf.gov/webservice/services/DAVIDWebService.DAVIDWebServiceHttpSoap11Endpoint/)
message:
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ns0="http://service.session.sample" xmlns:ns1="http://schemas.xmlsoap.org/soap/envelope/">
   <SOAP-ENV:Header/>
   <ns1:Body>
      <ns0:getTermClusterReport>
         <ns0:args0>3</ns0:args0>
         <ns0:args1>3</ns0:args1>
         <ns0:args2>3</ns0:args2>
         <ns0:args3>0.5</ns0:args3

Total clusters: 6


In [6]:
gene_id_map_reverse = {entry['To']: entry['From'] for _, entry in convert_table.iterrows()}
def map_gene_id_reverse(gene_name):
    try:
        return gene_id_map_reverse[gene_name]
    except:
        return gene_name
def map_gene_id_reverse_list(gene_list):
    try:
        return ','.join([map_gene_id_reverse(gene) for gene in gene_list.split(', ')])
    except:
        return gene_list

def format_pvalue(pvalue):
    try:
        if isinstance(pvalue, str):
            return '%.1e' % float(pvalue)
        else:
            return pvalue
    except:
        return pvalue

def format_percent(value):
    try:
        if isinstance(value, str):
            return int(float(value))
        else:
            return value
    except:
        return value

def filt_fun(pvalue):
    try:
        return float(pvalue) > 0.05
    except:
        return False

In [7]:
with pd.ExcelWriter(data_folder + file_name + '_david.xlsx') as writer:  
    for ii in range(num_list):
        cur_pd = pd.read_csv(save_folder + 'list_%d.csv' % ii, sep='\t', names=['Category', 'Term', 'Count', '%', 'Pvalue', 'Genes', 'List Total', 'Pop Hits', 'Pop Total', 'Fold Enrichment', 'Bonferroni', 'Benjamini', 'FDR'])
        filt1 = cur_pd['Category'] == 'UP_SEQ_FEATURE'
        filt2 = cur_pd['Pvalue'].apply(filt_fun)
        cur_pd = cur_pd[~(filt1 | filt2)]

        cur_pd['Term'] = cur_pd['Term'].apply(lambda x: x.split('~')[- 1])
        cur_pd['Genes'] = cur_pd['Genes'].apply(map_gene_id_reverse_list)
        cur_pd['Pvalue'] = cur_pd['Pvalue'].apply(format_pvalue)
        cur_pd['%'] = cur_pd['%'].apply(format_percent)
        
        ind_insert = np.where(cur_pd['Category'].str.startswith('Annotation Cluster'))[0]
        if len(ind_insert) > 1:
            for kk, ind in enumerate(ind_insert[1: ]):
                cur_pd = pd.DataFrame(np.insert(cur_pd.values, ind + kk, values=[" "] * len(cur_pd.columns), axis=0),columns = cur_pd.columns)

        cur_pd.to_excel(writer, sheet_name='program_%d' % ii, index=False)
        writer.sheets['program_%d' % ii].set_column(0, 0, 25)
        writer.sheets['program_%d' % ii].set_column(1, 1, 50)
        writer.sheets['program_%d' % ii].set_column(2, 2, 5)
        writer.sheets['program_%d' % ii].set_column(3, 3, 5)
        writer.sheets['program_%d' % ii].set_column(5, 5, 100)

In [9]:
with pd.ExcelWriter(data_folder + file_name + '_david_full.xlsx') as writer:  
    for ii in range(num_list):
        cur_pd = pd.read_csv(save_folder + 'list_%d.csv' % ii, sep='\t', names=['Category', 'Term', 'Count', '%', 'Pvalue', 'Genes', 'List Total', 'Pop Hits', 'Pop Total', 'Fold Enrichment', 'Bonferroni', 'Benjamini', 'FDR'])

        cur_pd['Term'] = cur_pd['Term'].apply(lambda x: x.split('~')[- 1])
        cur_pd['Genes'] = cur_pd['Genes'].apply(map_gene_id_reverse_list)
        cur_pd['Pvalue'] = cur_pd['Pvalue'].apply(format_pvalue)
        cur_pd['%'] = cur_pd['%'].apply(format_percent)
        
        ind_insert = np.where(cur_pd['Category'].str.startswith('Annotation Cluster'))[0]
        if len(ind_insert) > 1:
            for kk, ind in enumerate(ind_insert[1: ]):
                cur_pd = pd.DataFrame(np.insert(cur_pd.values, ind + kk, values=[" "] * len(cur_pd.columns), axis=0),columns = cur_pd.columns)

        cur_pd.to_excel(writer, sheet_name='program_%d' % ii, index=False)
        writer.sheets['program_%d' % ii].set_column(0, 0, 25)
        writer.sheets['program_%d' % ii].set_column(1, 1, 50)
        writer.sheets['program_%d' % ii].set_column(2, 2, 5)
        writer.sheets['program_%d' % ii].set_column(3, 3, 5)
        writer.sheets['program_%d' % ii].set_column(5, 5, 100)